### ¿Cuál es la puntuación de satisfacción laboral esperada para un empleado, considerando el balance trabajo-vida, la reputación de la empresa y las oportunidades de liderazgo?

* Características: Balance Trabajo-Vida, Reputación de la Empresa y Oportunidadez de Liderazgo
* Clasificacion: Satisfaccion de Trabajo

#### Cargar y dividir datos

In [2]:
'''
import pandas as pd
from sklearn.model_selection import train_test_split

# Cargar el archivo CSV
archivo = 'archivo_estandarizado_normalizado.csv'
df = pd.read_csv(archivo)

# Selección de las características (X) y la variable objetivo (Y)
X = df[['Horas Extras+norm', 'Oportunidades de Innovación+norm', 'Nivel de Trabajo+norm']]
Y = df['Satisfacción del Trabajo+norm']

# Divide el 80% para entrenamiento y 20% para validación/prueba
X_train, X_temp, Y_train, Y_temp = train_test_split(X, Y, test_size=0.2, random_state=42)

# Divide el 20% restante en partes iguales para validación y prueba (10% y 10%)
X_val, X_test, Y_val, Y_test = train_test_split(X_temp, Y_temp, test_size=0.5, random_state=42)

print(f"Tamaño de X_train: {X_train.shape}, X_val: {X_val.shape}, X_test: {X_test.shape}")
'''
import pandas as pd
import numpy as np
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler

# Cargar el archivo CSV
archivo = 'archivo_estandarizado_normalizado.csv'
df = pd.read_csv(archivo)

from sklearn.model_selection import train_test_split

# Selección de columnas de interés para la predicción de la "Satisfacción del Trabajo"
target_column = 'Satisfacción del Trabajo'

# Variables de interés en versiones estandarizadas
feature_columns_estandarizadas = [
    'Balance Trabajo-Vida+estand',
    'Reputación de la Empresa+estand',
    'Oportunidades de Liderazgo+estand'
]

# Variables de interés en versiones normalizadas
feature_columns_normalizadas = [
    'Balance Trabajo-Vida+norm',
    'Reputación de la Empresa+norm',
    'Oportunidades de Liderazgo+norm'
]

# Definir las variables predictoras y la variable objetivo
X_estandarizado = df[feature_columns_estandarizadas]
X_normalizado = df[feature_columns_normalizadas]
y = df[target_column]

# Dividir los datos en entrenamiento (80%) y prueba (20%) para ambas versiones
X_train_est, X_test_est, y_train, y_test = train_test_split(X_estandarizado, y, test_size=0.2, random_state=42)
X_train_norm, X_test_norm, _, _ = train_test_split(X_normalizado, y, test_size=0.2, random_state=42)

# Verificar las formas de los conjuntos de entrenamiento y prueba
X_train_est.shape, X_test_est.shape, X_train_norm.shape, X_test_norm.shape, y_train.shape, y_test.shape


((50214, 3), (12554, 3), (50214, 3), (12554, 3), (50214,), (12554,))

#### Entrenamiento con SVM

In [3]:
from sklearn.linear_model import LinearRegression
from sklearn.metrics import r2_score, mean_squared_error, mean_absolute_error
import numpy as np

# Inicializar el modelo de Regresión Lineal
linear_model_est = LinearRegression()
linear_model_norm = LinearRegression()

# Entrenar el modelo en los datos estandarizados
linear_model_est.fit(X_train_est, y_train)

# Entrenar el modelo en los datos normalizados
linear_model_norm.fit(X_train_norm, y_train)

# Hacer predicciones en el conjunto de prueba
y_pred_est = linear_model_est.predict(X_test_est)
y_pred_norm = linear_model_norm.predict(X_test_norm)

# Calcular las métricas para el conjunto estandarizado
r2_est = r2_score(y_test, y_pred_est)
rmse_est = np.sqrt(mean_squared_error(y_test, y_pred_est))
mae_est = mean_absolute_error(y_test, y_pred_est)

# Calcular las métricas para el conjunto normalizado
r2_norm = r2_score(y_test, y_pred_norm)
rmse_norm = np.sqrt(mean_squared_error(y_test, y_pred_norm))
mae_norm = mean_absolute_error(y_test, y_pred_norm)

# Resultados
{
    'Estandarizado': {'R2': r2_est, 'RMSE': rmse_est, 'MAE': mae_est},
    'Normalizado': {'R2': r2_norm, 'RMSE': rmse_norm, 'MAE': mae_norm}
}



{'Estandarizado': {'R2': 0.0002959474023764841,
  'RMSE': np.float64(0.9684200151778256),
  'MAE': np.float64(0.789577699521196)},
 'Normalizado': {'R2': 0.0002959474023764841,
  'RMSE': np.float64(0.9684200151778256),
  'MAE': np.float64(0.7895776995211962)}}

In [4]:
from sklearn.ensemble import RandomForestRegressor
from sklearn.neighbors import KNeighborsRegressor

# Inicializar los modelos
rf_model_est = RandomForestRegressor(random_state=42)
rf_model_norm = RandomForestRegressor(random_state=42)
knn_model_est = KNeighborsRegressor()
knn_model_norm = KNeighborsRegressor()

# Entrenar los modelos en los datos estandarizados
rf_model_est.fit(X_train_est, y_train)
knn_model_est.fit(X_train_est, y_train)

# Entrenar los modelos en los datos normalizados
rf_model_norm.fit(X_train_norm, y_train)
knn_model_norm.fit(X_train_norm, y_train)

# Hacer predicciones en el conjunto de prueba para Bosques Aleatorios
y_pred_rf_est = rf_model_est.predict(X_test_est)
y_pred_rf_norm = rf_model_norm.predict(X_test_norm)

# Hacer predicciones en el conjunto de prueba para K-Vecinos Más Cercanos
y_pred_knn_est = knn_model_est.predict(X_test_est)
y_pred_knn_norm = knn_model_norm.predict(X_test_norm)

# Calcular las métricas para el modelo de Bosques Aleatorios en ambos conjuntos
r2_rf_est = r2_score(y_test, y_pred_rf_est)
rmse_rf_est = np.sqrt(mean_squared_error(y_test, y_pred_rf_est))
mae_rf_est = mean_absolute_error(y_test, y_pred_rf_est)

r2_rf_norm = r2_score(y_test, y_pred_rf_norm)
rmse_rf_norm = np.sqrt(mean_squared_error(y_test, y_pred_rf_norm))
mae_rf_norm = mean_absolute_error(y_test, y_pred_rf_norm)

# Calcular las métricas para el modelo de K-Vecinos Más Cercanos en ambos conjuntos
r2_knn_est = r2_score(y_test, y_pred_knn_est)
rmse_knn_est = np.sqrt(mean_squared_error(y_test, y_pred_knn_est))
mae_knn_est = mean_absolute_error(y_test, y_pred_knn_est)

r2_knn_norm = r2_score(y_test, y_pred_knn_norm)
rmse_knn_norm = np.sqrt(mean_squared_error(y_test, y_pred_knn_norm))
mae_knn_norm = mean_absolute_error(y_test, y_pred_knn_norm)

# Resultados
{
    'Random Forest Estandarizado': {'R2': r2_rf_est, 'RMSE': rmse_rf_est, 'MAE': mae_rf_est},
    'Random Forest Normalizado': {'R2': r2_rf_norm, 'RMSE': rmse_rf_norm, 'MAE': mae_rf_norm},
    'KNN Estandarizado': {'R2': r2_knn_est, 'RMSE': rmse_knn_est, 'MAE': mae_knn_est},
    'KNN Normalizado': {'R2': r2_knn_norm, 'RMSE': rmse_knn_norm, 'MAE': mae_knn_norm}
}


{'Random Forest Estandarizado': {'R2': 0.0012046791073027086,
  'RMSE': np.float64(0.9679797678626253),
  'MAE': np.float64(0.78712556059863)},
 'Random Forest Normalizado': {'R2': 0.0012046791073027086,
  'RMSE': np.float64(0.9679797678626253),
  'MAE': np.float64(0.78712556059863)},
 'KNN Estandarizado': {'R2': -0.15795277030511534,
  'RMSE': np.float64(1.0422537191293832),
  'MAE': np.float64(0.8604588179066434)},
 'KNN Normalizado': {'R2': -0.2860896305248686,
  'RMSE': np.float64(1.0984079025317113),
  'MAE': np.float64(0.8894694917954437)}}

### Entrenamiento con Bosques Aleatorios

In [13]:
import pandas as pd
import numpy as np
from sklearn.model_selection import train_test_split
from sklearn.ensemble import RandomForestRegressor
from sklearn.neighbors import KNeighborsRegressor
from sklearn.linear_model import LinearRegression
from sklearn.feature_selection import SelectKBest, f_regression
from sklearn.metrics import r2_score, mean_squared_error, mean_absolute_error

# Cargar el conjunto de datos
file_path = 'archivo_EN.csv'  # Cambia esto si el archivo está en otra ubicación
df = pd.read_csv(file_path)

# Filtrar las columnas que terminan en '+norm' y '+estand'
norm_columns = [col for col in df.columns if col.endswith('+norm')]
estand_columns = [col for col in df.columns if col.endswith('+estand')]

# Dividir los datos en variables predictoras (X) y variables objetivo (y) para cada caso
# Para la predicción de Satisfacción de Trabajo+norm
X_norm = df.drop(columns=norm_columns)
y_norm = df[norm_columns[0]]  # Asumimos que solo hay una columna de 'Satisfacción de Trabajo+norm'

# Para la predicción de Satisfacción de Trabajo+estand
X_estand = df.drop(columns=estand_columns)
y_estand = df[estand_columns[0]]  # Asumimos que solo hay una columna de 'Satisfacción de Trabajo+estand'

# Dividir los datos en conjuntos de entrenamiento y prueba (80% - 20%) para cada caso
X_train_norm, X_test_norm, y_train_norm, y_test_norm = train_test_split(X_norm, y_norm, test_size=0.2, random_state=42)
X_train_estand, X_test_estand, y_train_estand, y_test_estand = train_test_split(X_estand, y_estand, test_size=0.2, random_state=42)

# Función para seleccionar las mejores características con SelectKBest
def select_best_features(X_train, X_test, y_train, k=10):
    selector = SelectKBest(score_func=f_regression, k=k)  # Usamos f_regression para evaluar la relevancia
    X_train_selected = selector.fit_transform(X_train, y_train)
    X_test_selected = selector.transform(X_test)
    
    # Obtener las mejores características y sus puntuaciones
    best_feature_indices = selector.get_support(indices=True)
    best_features = X_train.columns[best_feature_indices]
    feature_scores = selector.scores_[best_feature_indices]
    
    # Crear un DataFrame con las mejores características y sus puntuaciones
    best_features_df = pd.DataFrame({
        'Feature': best_features,
        'Score': feature_scores
    }).sort_values(by='Score', ascending=False)
    
    return X_train_selected, X_test_selected, best_features_df

# Seleccionar las mejores características para ambos casos (norm y estand)
X_train_norm_selected, X_test_norm_selected, best_features_norm_df = select_best_features(X_train_norm, X_test_norm, y_train_norm)
X_train_estand_selected, X_test_estand_selected, best_features_estand_df = select_best_features(X_train_estand, X_test_estand, y_train_estand)

# Mostrar las mejores características seleccionadas para cada caso
print("Mejores características para Satisfacción de Trabajo+norm y sus puntuaciones:")
print(best_features_norm_df)

print("\nMejores características para Satisfacción de Trabajo+estand y sus puntuaciones:")
print(best_features_estand_df)

# Modelos de regresión
models = {
    'RandomForestRegressor': RandomForestRegressor(random_state=42),
    'KNeighborsRegressor': KNeighborsRegressor(),
    'LinearRegression': LinearRegression()
}

# Función para evaluar los modelos
def evaluate_models(models, X_train, X_test, y_train, y_test, X_train_selected, X_test_selected):
    results = {}
    for model_name, model in models.items():
        # Entrenamiento con todas las variables
        model.fit(X_train, y_train)
        y_pred = model.predict(X_test)
        results[model_name] = {
            'r2_all': r2_score(y_test, y_pred),  # R2 para medir la proporción de variación explicada
            'rmse_all': mean_squared_error(y_test, y_pred, squared=False),  # RMSE para medir el error promedio
            'mae_all': mean_absolute_error(y_test, y_pred)  # MAE para medir el error absoluto promedio
        }

        # Entrenamiento con selección de características
        model.fit(X_train_selected, y_train)
        y_pred_selected = model.predict(X_test_selected)
        results[model_name].update({
            'r2_selected': r2_score(y_test, y_pred_selected),  # R2 para las variables seleccionadas
            'rmse_selected': mean_squared_error(y_test, y_pred_selected, squared=False),
            'mae_selected': mean_absolute_error(y_test, y_pred_selected)
        })
    
    return results

# Evaluar los modelos para Satisfacción de Trabajo+norm
results_norm = evaluate_models(models, X_train_norm, X_test_norm, y_train_norm, y_test_norm, X_train_norm_selected, X_test_norm_selected)

# Evaluar los modelos para Satisfacción de Trabajo+estand
results_estand = evaluate_models(models, X_train_estand, X_test_estand, y_train_estand, y_test_estand, X_train_estand_selected, X_test_estand_selected)

# Mostrar resultados para Satisfacción de Trabajo+norm
print("\nResultados para Satisfacción de Trabajo+norm:")
for model_name, metrics in results_norm.items():
    print(f"\nModelo: {model_name}")
    print("\nResultados con todas las variables:")
    print(f"R2: {metrics['r2_all']:.4f}")
    print(f"RMSE: {metrics['rmse_all']:.4f}")
    print(f"MAE: {metrics['mae_all']:.4f}")
    
    print("\nResultados con selección de las mejores variables:")
    print(f"R2: {metrics['r2_selected']:.4f}")
    print(f"RMSE: {metrics['rmse_selected']:.4f}")
    print(f"MAE: {metrics['mae_selected']:.4f}")

# Mostrar resultados para Satisfacción de Trabajo+estand
print("\nResultados para Satisfacción de Trabajo+estand:")
for model_name, metrics in results_estand.items():
    print(f"\nModelo: {model_name}")
    print("\nResultados con todas las variables:")
    print(f"R2: {metrics['r2_all']:.4f}")
    print(f"RMSE: {metrics['rmse_all']:.4f}")
    print(f"MAE: {metrics['mae_all']:.4f}")
    
    print("\nResultados con selección de las mejores variables:")
    print(f"R2: {metrics['r2_selected']:.4f}")
    print(f"RMSE: {metrics['rmse_selected']:.4f}")
    print(f"MAE: {metrics['mae_selected']:.4f}")


Mejores características para Satisfacción de Trabajo+norm y sus puntuaciones:
                               Feature         Score
1                          Edad+estand  5.462192e+17
2      Antigüedad en la Empresa+estand  1.690627e+04
7  Meses desde el último evento+estand  2.529194e+03
0                          ID Empleado  1.906651e+03
3                Rol de Trabajo+estand  1.746140e+03
6             Tamaño de Empresa+estand  1.060711e+03
4               Ingreso Mensual+estand  9.291019e+02
5              Distancia a Casa+estand  7.860848e+02
9      Reputación de la Empresa+estand  2.527854e+02
8                Trabajo Remoto+estand  1.571239e+02

Mejores características para Satisfacción de Trabajo+estand y sus puntuaciones:
                             Feature         Score
1                          Edad+norm  2.737709e+17
2      Antigüedad en la Empresa+norm  1.690627e+04
7  Meses desde el último evento+norm  2.529194e+03
0                        ID Empleado  1.906651e+03
3  

c:\General\Cuarto\2°do Cuatrimestre\Machine Learning\EDA\myenv\Lib\site-packages\sklearn\metrics\_regression.py:492: FutureWarning: 'squared' is deprecated in version 1.4 and will be removed in 1.6. To calculate the root mean squared error, use the function'root_mean_squared_error'.
  warnings.warn(
c:\General\Cuarto\2°do Cuatrimestre\Machine Learning\EDA\myenv\Lib\site-packages\sklearn\metrics\_regression.py:492: FutureWarning: 'squared' is deprecated in version 1.4 and will be removed in 1.6. To calculate the root mean squared error, use the function'root_mean_squared_error'.
  warnings.warn(
c:\General\Cuarto\2°do Cuatrimestre\Machine Learning\EDA\myenv\Lib\site-packages\sklearn\metrics\_regression.py:492: FutureWarning: 'squared' is deprecated in version 1.4 and will be removed in 1.6. To calculate the root mean squared error, use the function'root_mean_squared_error'.
  warnings.warn(
c:\General\Cuarto\2°do Cuatrimestre\Machine Learning\EDA\myenv\Lib\site-packages\sklearn\metrics\


Resultados para Satisfacción de Trabajo+norm:

Modelo: RandomForestRegressor

Resultados con todas las variables:
R2: 1.0000
RMSE: 0.0000
MAE: 0.0000

Resultados con selección de las mejores variables:
R2: 1.0000
RMSE: 0.0000
MAE: 0.0000

Modelo: KNeighborsRegressor

Resultados con todas las variables:
R2: 0.0480
RMSE: 0.2454
MAE: 0.1968

Resultados con selección de las mejores variables:
R2: 0.0306
RMSE: 0.2476
MAE: 0.1989

Modelo: LinearRegression

Resultados con todas las variables:
R2: 1.0000
RMSE: 0.0000
MAE: 0.0000

Resultados con selección de las mejores variables:
R2: 1.0000
RMSE: 0.0000
MAE: 0.0000

Resultados para Satisfacción de Trabajo+estand:

Modelo: RandomForestRegressor

Resultados con todas las variables:
R2: 1.0000
RMSE: 0.0000
MAE: 0.0000

Resultados con selección de las mejores variables:
R2: 1.0000
RMSE: 0.0000
MAE: 0.0000

Modelo: KNeighborsRegressor

Resultados con todas las variables:
R2: -0.0302
RMSE: 1.0047
MAE: 0.8075

Resultados con selección de las mejores

c:\General\Cuarto\2°do Cuatrimestre\Machine Learning\EDA\myenv\Lib\site-packages\sklearn\metrics\_regression.py:492: FutureWarning: 'squared' is deprecated in version 1.4 and will be removed in 1.6. To calculate the root mean squared error, use the function'root_mean_squared_error'.
  warnings.warn(


### Entrenamiento con Regresión Logística

In [ ]:
import pandas as pd
import numpy as np
from sklearn.model_selection import train_test_split
from sklearn.linear_model import LogisticRegression  # Usamos LogisticRegression para Softmax
from sklearn.feature_selection import SelectKBest, f_classif  # f_classif para clasificación
from sklearn.metrics import accuracy_score, classification_report, confusion_matrix

# Cargar el conjunto de datos
file_path = 'archivo_estandarizado_normalizado.csv'  # Cambia esto si el archivo está en otra ubicación
df = pd.read_csv(file_path)

# Filtrar las columnas que terminan en '+norm' y '+estand'
norm_columns = [col for col in df.columns if col.endswith('+norm')]
estand_columns = [col for col in df.columns if col.endswith('+estand')]

# Dividir los datos en variables predictoras (X) y variables objetivo (y) para cada caso
# Para la predicción de Satisfacción de Trabajo+norm
X_norm = df.drop(columns=norm_columns)
y_norm = df[norm_columns[0]]  # Asumimos que solo hay una columna de 'Satisfacción de Trabajo+norm'

# Para la predicción de Satisfacción de Trabajo+estand
X_estand = df.drop(columns=estand_columns)
y_estand = df[estand_columns[0]]  # Asumimos que solo hay una columna de 'Satisfacción de Trabajo+estand'

# Dividir los datos en conjuntos de entrenamiento y prueba (80% - 20%) para cada caso
X_train_norm, X_test_norm, y_train_norm, y_test_norm = train_test_split(X_norm, y_norm, test_size=0.2, random_state=42)
X_train_estand, X_test_estand, y_train_estand, y_test_estand = train_test_split(X_estand, y_estand, test_size=0.2, random_state=42)

# Función para seleccionar las mejores características con SelectKBest
def select_best_features(X_train, X_test, y_train, k=10):
    selector = SelectKBest(score_func=f_classif, k=k)  # Usamos f_classif para clasificación
    X_train_selected = selector.fit_transform(X_train, y_train)
    X_test_selected = selector.transform(X_test)
    
    # Obtener las mejores características y sus puntuaciones
    best_feature_indices = selector.get_support(indices=True)
    best_features = X_train.columns[best_feature_indices]
    feature_scores = selector.scores_[best_feature_indices]
    
    # Crear un DataFrame con las mejores características y sus puntuaciones
    best_features_df = pd.DataFrame({
        'Feature': best_features,
        'Score': feature_scores
    }).sort_values(by='Score', ascending=False)
    
    return X_train_selected, X_test_selected, best_features_df

# Seleccionar las mejores características para ambos casos (norm y estand)
X_train_norm_selected, X_test_norm_selected, best_features_norm_df = select_best_features(X_train_norm, X_test_norm, y_train_norm)
X_train_estand_selected, X_test_estand_selected, best_features_estand_df = select_best_features(X_train_estand, X_test_estand, y_train_estand)

# Mostrar las mejores características seleccionadas para cada caso
print("Mejores características para Satisfacción de Trabajo+norm y sus puntuaciones:")
print(best_features_norm_df)

print("\nMejores características para Satisfacción de Trabajo+estand y sus puntuaciones:")
print(best_features_estand_df)

# Modelos de clasificación (Softmax a través de LogisticRegression)
models = {
    'LogisticRegression (Softmax)': LogisticRegression(max_iter=1000, multi_class='multinomial', solver='lbfgs', random_state=42)
}

# Función para evaluar los modelos
def evaluate_models(models, X_train, X_test, y_train, y_test, X_train_selected, X_test_selected):
    results = {}
    for model_name, model in models.items():
        # Entrenamiento con todas las variables
        model.fit(X_train, y_train)
        y_pred = model.predict(X_test)
        results[model_name] = {
            'accuracy_all': accuracy_score(y_test, y_pred),  # Accuracy para clasificación
            'classification_report_all': classification_report(y_test, y_pred),  # Precision, Recall, F1-score
            'confusion_matrix_all': confusion_matrix(y_test, y_pred)  # Matriz de confusión
        }

        # Entrenamiento con selección de características
        model.fit(X_train_selected, y_train)
        y_pred_selected = model.predict(X_test_selected)
        results[model_name].update({
            'accuracy_selected': accuracy_score(y_test, y_pred_selected),  # Accuracy para las variables seleccionadas
            'classification_report_selected': classification_report(y_test, y_pred_selected),
            'confusion_matrix_selected': confusion_matrix(y_test, y_pred_selected)
        })
    
    return results

# Evaluar los modelos para Satisfacción de Trabajo+norm
results_norm = evaluate_models(models, X_train_norm, X_test_norm, y_train_norm, y_test_norm, X_train_norm_selected, X_test_norm_selected)

# Evaluar los modelos para Satisfacción de Trabajo+estand
results_estand = evaluate_models(models, X_train_estand, X_test_estand, y_train_estand, y_test_estand, X_train_estand_selected, X_test_estand_selected)

# Mostrar resultados para Satisfacción de Trabajo+norm
print("\nResultados para Satisfacción de Trabajo+norm:")
for model_name, metrics in results_norm.items():
    print(f"\nModelo: {model_name}")
    print("\nResultados con todas las variables:")
    print(f"Accuracy: {metrics['accuracy_all']:.4f}")
    print(f"Reporte de clasificación:\n{metrics['classification_report_all']}")
    print(f"Matriz de confusión:\n{metrics['confusion_matrix_all']}")
    
    print("\nResultados con selección de las mejores variables:")
    print(f"Accuracy: {metrics['accuracy_selected']:.4f}")
    print(f"Reporte de clasificación:\n{metrics['classification_report_selected']}")
    print(f"Matriz de confusión:\n{metrics['confusion_matrix_selected']}")

# Mostrar resultados para Satisfacción de Trabajo+estand
print("\nResultados para Satisfacción de Trabajo+estand:")
for model_name, metrics in results_estand.items():
    print(f"\nModelo: {model_name}")
    print("\nResultados con todas las variables:")
    print(f"Accuracy: {metrics['accuracy_all']:.4f}")
    print(f"Reporte de clasificación:\n{metrics['classification_report_all']}")
    print(f"Matriz de confusión:\n{metrics['confusion_matrix_all']}")
    
    print("\nResultados con selección de las mejores variables:")
    print(f"Accuracy: {metrics['accuracy_selected']:.4f}")
    print(f"Reporte de clasificación:\n{metrics['classification_report_selected']}")
    print(f"Matriz de confusión:\n{metrics['confusion_matrix_selected']}")



Mejores características para Satisfacción de Trabajo+norm y sus puntuaciones:
                               Feature         Score
1                                 Edad  1.413342e+19
5                          Edad+estand  5.462192e+17
6      Antigüedad en la Empresa+estand  1.690627e+04
2             Antigüedad en la Empresa  1.690627e+04
9  Meses desde el último evento+estand  2.529194e+03
4         Meses desde el último evento  2.529194e+03
0                          ID Empleado  1.906651e+03
3                       Rol de Trabajo  1.746140e+03
7                Rol de Trabajo+estand  1.746140e+03
8             Tamaño de Empresa+estand  1.060711e+03

Mejores características para Satisfacción de Trabajo+estand y sus puntuaciones:
                             Feature         Score
1                               Edad  1.130674e+20
6                          Edad+norm  2.737709e+17
7      Antigüedad en la Empresa+norm  1.690627e+04
2           Antigüedad en la Empresa  1.690627e+04
9  

KeyboardInterrupt: 

### Comparación de todos los Resultados:

In [10]:
import pandas as pd
import numpy as np
from sklearn.model_selection import train_test_split
from sklearn.feature_selection import SelectKBest, f_classif  # f_classif para clasificación
from sklearn.metrics import accuracy_score, classification_report, confusion_matrix
from sklearn.neighbors import KNeighborsClassifier  # KNN
from sklearn.ensemble import RandomForestClassifier  # Random Forest
from sklearn.linear_model import LogisticRegression  # Algoritmo adicional: Regresión Logística
from sklearn.preprocessing import LabelEncoder  # Para la codificación de etiquetas

# Cargar el conjunto de datos
file_path = 'archivo_int64.csv'  # Cambia esto si el archivo está en otra ubicación
df = pd.read_csv(file_path)

# Filtrar las columnas que terminan en '+norm' y '+estand'
norm_columns = [col for col in df.columns if col.endswith('+norm')]
estand_columns = [col for col in df.columns if col.endswith('+estand')]

# Eliminar las columnas '+norm', '+estand' y 'ID Empleado' para obtener solo las variables relevantes
columns_to_drop = norm_columns + estand_columns + ['ID Empleado']
df_cleaned = df.drop(columns=columns_to_drop)

# Asegurarse de que la columna 'Satisfacción del Trabajo' exista
target_column = 'Satisfacción del Trabajo'
if target_column not in df_cleaned.columns:
    raise ValueError(f"La columna '{target_column}' no se encuentra en el DataFrame")

# Dividir los datos en variables predictoras (X) y la variable objetivo 'Satisfacción del Trabajo'
X = df_cleaned.drop(columns=[target_column])  # Todas las columnas excepto 'Satisfacción del Trabajo'
y = df_cleaned[target_column]  # 'Satisfacción del Trabajo' como la variable objetivo

# Convertir las etiquetas de 'Satisfacción del Trabajo' en valores numéricos enteros para clasificación
label_encoder = LabelEncoder()
y_encoded = label_encoder.fit_transform(y)

# Dividir los datos en conjuntos de entrenamiento y prueba (80% - 20%)
X_train, X_test, y_train, y_test = train_test_split(X, y_encoded, test_size=0.2, random_state=42)

# Variable para controlar el número de características a seleccionar
k_best_features = 8  # Modifica este valor según la cantidad de características que quieras usar

# Función para seleccionar las mejores características con SelectKBest
def select_best_features(X_train, X_test, y_train, k=10):
    selector = SelectKBest(score_func=f_classif, k=k)  # Usamos f_classif para clasificación
    X_train_selected = selector.fit_transform(X_train, y_train)
    X_test_selected = selector.transform(X_test)
    
    # Obtener las mejores características y sus puntuaciones
    best_feature_indices = selector.get_support(indices=True)
    best_features = X_train.columns[best_feature_indices]
    feature_scores = selector.scores_[best_feature_indices]
    
    # Crear un DataFrame con las mejores características y sus puntuaciones
    best_features_df = pd.DataFrame({
        'Feature': best_features,
        'Score': feature_scores
    }).sort_values(by='Score', ascending=False)
    
    return X_train_selected, X_test_selected, best_features_df

# Seleccionar las mejores características
X_train_selected, X_test_selected, best_features_df = select_best_features(X_train, X_test, y_train, k=k_best_features)

# Mostrar las mejores características seleccionadas
print("Mejores características para predecir Satisfacción del Trabajo y sus puntuaciones:")
print(best_features_df)

# Función para evaluar los modelos
def evaluate_models(X_train, X_test, y_train, y_test, X_train_selected, X_test_selected):
    results = {}
    
    # Inicializar los modelos (KNN, Random Forest y Regresión Logística)
    knn_model = KNeighborsClassifier(n_neighbors=5)  # Modelo KNN
    rf_model = RandomForestClassifier(n_estimators=100, random_state=42)  # Modelo Random Forest
    lr_model = LogisticRegression(random_state=42, max_iter=1000)  # Aumentar max_iter para evitar advertencias
    
    # Evaluar el modelo KNN con todas las características
    knn_model.fit(X_train, y_train)
    y_pred_knn = knn_model.predict(X_test)
    results['KNN_all'] = {
        'accuracy_all': accuracy_score(y_test, y_pred_knn),
        'classification_report_all': classification_report(y_test, y_pred_knn),
        'confusion_matrix_all': confusion_matrix(y_test, y_pred_knn)
    }

    # Evaluar el modelo Random Forest con todas las características
    rf_model.fit(X_train, y_train)
    y_pred_rf = rf_model.predict(X_test)
    results['RF_all'] = {
        'accuracy_all': accuracy_score(y_test, y_pred_rf),
        'classification_report_all': classification_report(y_test, y_pred_rf),
        'confusion_matrix_all': confusion_matrix(y_test, y_pred_rf)
    }

    # Evaluar el modelo Regresión Logística con todas las características
    lr_model.fit(X_train, y_train)
    y_pred_lr = lr_model.predict(X_test)
    results['LR_all'] = {
        'accuracy_all': accuracy_score(y_test, y_pred_lr),
        'classification_report_all': classification_report(y_test, y_pred_lr),
        'confusion_matrix_all': confusion_matrix(y_test, y_pred_lr)
    }

    # Evaluación con selección de características para KNN
    knn_model.fit(X_train_selected, y_train)
    y_pred_knn_selected = knn_model.predict(X_test_selected)
    results['KNN_selected'] = {
        'accuracy_selected': accuracy_score(y_test, y_pred_knn_selected),
        'classification_report_selected': classification_report(y_test, y_pred_knn_selected),
        'confusion_matrix_selected': confusion_matrix(y_test, y_pred_knn_selected)
    }

    # Evaluación con selección de características para Random Forest
    rf_model.fit(X_train_selected, y_train)
    y_pred_rf_selected = rf_model.predict(X_test_selected)
    results['RF_selected'] = {
        'accuracy_selected': accuracy_score(y_test, y_pred_rf_selected),
        'classification_report_selected': classification_report(y_test, y_pred_rf_selected),
        'confusion_matrix_selected': confusion_matrix(y_test, y_pred_rf_selected)
    }

    # Evaluación con selección de características para Regresión Logística
    lr_model.fit(X_train_selected, y_train)
    y_pred_lr_selected = lr_model.predict(X_test_selected)
    results['LR_selected'] = {
        'accuracy_selected': accuracy_score(y_test, y_pred_lr_selected),
        'classification_report_selected': classification_report(y_test, y_pred_lr_selected),
        'confusion_matrix_selected': confusion_matrix(y_test, y_pred_lr_selected)
    }
    
    return results

# Evaluar los resultados
results = evaluate_models(X_train, X_test, y_train, y_test, X_train_selected, X_test_selected)

# Mostrar resultados
print("\nResultados para clasificación:")
for model_name, metrics in results.items():
    print(f"\nModelo: {model_name}")
    print(f"\nResultados con todas las variables:")
    if 'accuracy_all' in metrics:
        print(f"Accuracy: {metrics['accuracy_all']:.4f}")
        print(f"Reporte de clasificación:\n{metrics['classification_report_all']}")
        print(f"Matriz de confusión:\n{metrics['confusion_matrix_all']}")
    
    print("\nResultados con selección de las mejores variables:")
    if 'accuracy_selected' in metrics:
        print(f"Accuracy: {metrics['accuracy_selected']:.4f}")
        print(f"Reporte de clasificación:\n{metrics['classification_report_selected']}")
        print(f"Matriz de confusión:\n{metrics['confusion_matrix_selected']}")
    else:
        print("\nNo se encontró la evaluación con las mejores variables seleccionadas.")


Mejores características para predecir Satisfacción del Trabajo y sus puntuaciones:
                        Feature       Score
1                Rol de Trabajo  536.723052
3             Tamaño de Empresa  406.954854
2              Distancia a Casa  311.023650
7      Reputación de la Empresa  118.366184
4  Meses desde el último evento   86.156069
0      Antigüedad en la Empresa   76.708110
5                Trabajo Remoto   58.207989
6   Oportunidades de Innovación   49.239033


c:\General\Cuarto\2°do Cuatrimestre\Machine Learning\EDA\myenv\Lib\site-packages\sklearn\linear_model\_logistic.py:469: ConvergenceWarning: lbfgs failed to converge (status=1):
STOP: TOTAL NO. of ITERATIONS REACHED LIMIT.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
Please also refer to the documentation for alternative solver options:
    https://scikit-learn.org/stable/modules/linear_model.html#logistic-regression
  n_iter_i = _check_optimize_result(
c:\General\Cuarto\2°do Cuatrimestre\Machine Learning\EDA\myenv\Lib\site-packages\sklearn\metrics\_classification.py:1531: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", len(result))
c:\General\Cuarto\2°do Cuatrimestre\Machine Learning\EDA\myenv\Lib\site-packages\sklearn\m


Resultados para clasificación:

Modelo: KNN_all

Resultados con todas las variables:
Accuracy: 0.6091
Reporte de clasificación:
              precision    recall  f1-score   support

           0       0.66      0.59      0.62      1693
           1       0.50      0.47      0.49      2537
           2       0.60      0.73      0.65      5222
           3       0.72      0.54      0.62      3102

    accuracy                           0.61     12554
   macro avg       0.62      0.58      0.59     12554
weighted avg       0.62      0.61      0.61     12554

Matriz de confusión:
[[1002  136  483   72]
 [ 130 1197 1036  174]
 [ 282  762 3788  390]
 [ 115  289 1038 1660]]

Resultados con selección de las mejores variables:

No se encontró la evaluación con las mejores variables seleccionadas.

Modelo: RF_all

Resultados con todas las variables:
Accuracy: 0.6788
Reporte de clasificación:
              precision    recall  f1-score   support

           0       0.99      0.55      0.71     

c:\General\Cuarto\2°do Cuatrimestre\Machine Learning\EDA\myenv\Lib\site-packages\sklearn\metrics\_classification.py:1531: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", len(result))
c:\General\Cuarto\2°do Cuatrimestre\Machine Learning\EDA\myenv\Lib\site-packages\sklearn\metrics\_classification.py:1531: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", len(result))
c:\General\Cuarto\2°do Cuatrimestre\Machine Learning\EDA\myenv\Lib\site-packages\sklearn\metrics\_classification.py:1531: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.